In [29]:
import pandas as pd
from openpyxl import load_workbook

In [95]:
def header_row_number(sheet):
    for row in sheet.iter_rows(min_row=1, max_col=8):
        if row[1].value and row[1].value == 'Merkki':
            return row[1].row
    return None

COLS = {
    'Käyttöönottovuosi': 'model_year',
    'Käyttööottovuosi': 'model_year',
    'Merkki': 'brand',
    'Malli': 'model',
    'Hylkäys-%': 'fail_pct',
    'Hylkäys %': 'fail_pct',
    "Katsastusten lukumäärä": 'checked_cars',
    "Ajettujen kilometrien keskiarvo": 'driven_avg',
    "Ajettujen kilometrien mediaani": 'driven_med',
    "Yleisin hylkäyssyy": 'reason_1',
    "Toiseksi yleisin hylkäyssyy": 'reason_2',
    "Kolmanneksi yleisin hylkäyssyy": 'reason_3',
}

def should_keep(row):
    w = 'yhteensä'
    return not (w in str(row[0].value) or w in str(row[1].value) or w in str(row[2].value))

def clean_value(c):
    if not c:
        return None
    if isinstance(c.value, str):
        return c.value.replace("\n", " ")
    else:
        return c.value

for y in range(2016, 2020):
    print(y)
    xlsx = f'data/Henkilöautojen-määräaikaiskatsastusten-vuositilasto-{y}.xlsx'
    wb = load_workbook(filename = xlsx, read_only=True)
    sheet = wb.active
    header_row = header_row_number(sheet)
    table = [[clean_value(c) for c in r] \
                 for r in sheet.iter_rows(min_row=header_row, max_col=10) \
                     if should_keep(r)]
    
    df = pd.DataFrame(data=table[1:], columns=table[0])
    df.rename(inplace=True, columns=COLS)
    for k in df.keys():
        if k and k not in COLS.values():
            df.drop(inplace=True, columns=[k])
    df.dropna(inplace=True, axis=1, how='all')
    df = df.astype(dtype={'model_year': 'int16', 'brand': 'category', 'model': 'category', 'fail_pct': 'float16'})
    
    print(df.sample(5))
    print()


2016
      model_year       brand    model   fail_pct  \
1193        2004  Volkswagen  TOUAREG  26.953125   
45          2013       Skoda    RAPID   2.019531   
1213        2004       Honda   STREAM  29.406250   
698         2007      Toyota     AYGO   6.781250   
1354        2003        Seat   TOLEDO  35.843750   

                                reason_1                            reason_2  \
1193                         Käyttöjarru     Seisontajarrun dynamometritesti   
45                                  None                                None   
1213  Sisäinen valvontajärjestelmä (OBD)                         Käyttöjarru   
698                   Renkaat ja vanteet  Sisäinen valvontajärjestelmä (OBD)   
1354                         Käyttöjarru     Seisontajarrun dynamometritesti   

                           reason_3  
1193     Jousitus ja iskunvaimennus  
45                             None  
1213  Käyttöjarrun dynamometritesti  
698                    Etuakselisto  
1354       